In [ ]:
%pip install fastapi 
%pip install influxdb_client
%pip install minio

In [4]:
from base import LocalGateway, base_logger, PeriodicTrigger, BaseEventFabric, ExampleEventFabric

from influxdb_client import InfluxDBClient
from fastapi import FastAPI, Request
from datetime import datetime, timedelta
from minio import Minio
from io import BytesIO
import pandas as pd

# -------------------------- CONFIGURATION --------------------------

# InfluxDB credentials and database details
INFLUX_TOKEN = "http://192.168.1.132:8086"
INFLUX_ORG = "wise2024"
INFLUX_USER = "admin"
INFLUX_PASS = "secure_influx_iot_user"

VIZ_COMPONENT_URL = "http://192.168.1.132:9000"
SIF_SCHEDULER = ("SCH_SERVICE_NAME", "192.168.1.132:30032")

# Minio credentials and database details
MINIO_ENDPOINT = "192.168.1.132:9090"
MINIO_ACCESS_KEY = "peUyeVUBhKS7DvpFZgJu"
MINIO_SECRET_KEY = "J5VLWMfzNXBnhrm1kKHmO7DRbnU5XzqUO1iKWJfi"
MINIO_BUCKET = "models"

# Influx Buckets
BUCKETS = ["1_2_2", "1_2_7", "1_3_10", "1_3_11", "1_3_14", "1_4_12", "1_4_13"]  
BUCKETS_PIR = ["1_2_2", "1_3_11", "1_3_14", "1_4_13"]
BUCKET_CORRIDOR = "1_2_2"
BUCKET_BATHROOM = "1_3_11"
BUCKET_DOOR = "1_3_14"
BUCKET_BED = "1_4_13"

pir_buckets = {
        "1_2_2": "corridor",
        "1_3_11": "bathroom",
        "1_3_14": "door",
        "1_4_13": "bed",
    }

bucket_mapping = {
    "1_2_2": "corridor",
    "1_3_11": "bathroom",
    "1_3_14": "door",
    "1_4_13": "bed",
}

# ------------------------ INFLUXDB FUNCTIONS ------------------------

# Fetch data (inspired by the sif-viz-component fetch data structure)
def fetch_data(bucket, measurement, field):

    with InfluxDBClient(
        url=INFLUX_TOKEN, 
        org=INFLUX_ORG, 
        username=INFLUX_USER, 
        password=INFLUX_PASS, 
        verify_ssl=False) as client:
            p = {
                "_start": timedelta(days=-100),  # "fetch data starting from x days ago".
            }

            query_api = client.query_api()
            tables = query_api.query(f'''
                                    from(bucket: "{bucket}") |> range(start: _start)
                                    |> filter(fn: (r) => r["_measurement"] == "{measurement}")
                                    |> filter(fn: (r) => r["_type"] == "{"sensor-value"}")
                                    |> filter(fn: (r) => r["_field"] == "{field}")
                                    ''', params=p)          
            obj = []
            
            base_logger.info(tables)
            for table in tables:
                for record in table.records:
                    val = {}
                    base_logger.info(record)
                    val["bucket"] = bucket
                    val["timestamp"] = record["_time"].timestamp() * 1000
                    val["value"] = record["_value"]
                    if len(val.keys()) != 0:
                        obj.append(val)

            return obj

In [5]:
def fetch_data_arrangement():   # Makes easier readable values for the model 

    # Fetch data for each bucket individually
    bathroomfetch = fetch_data(BUCKET_BATHROOM, "PIR", "roomID")
    corridorfetch = fetch_data(BUCKET_CORRIDOR, "PIR", "roomID")
    bedfetch = fetch_data(BUCKET_BED, "PIR", "roomID")
    doorfetch = fetch_data(BUCKET_DOOR, "door", "roomID2")

    # Add bucket identifier to each fetched dataset
    for record in bathroomfetch:
        record["bucket"] = "1_3_11"  # BUCKET_BATHROOM
    for record in corridorfetch:
        record["bucket"] = "1_2_2"   # BUCKET_CORRIDOR
    for record in bedfetch:
        record["bucket"] = "1_4_13"  # BUCKET_BED
    for record in doorfetch:
        record["bucket"] = "1_3_14"  # BUCKET_DOOR

    # Merge all fetched data
    all_fetched = bathroomfetch + corridorfetch + bedfetch + doorfetch

    # Create a pandas DataFrame
    df = pd.DataFrame(all_fetched)
    print(all_fetched)
    # Map bucket to value
    df["value"] = df["bucket"].map(bucket_mapping)

    # Convert timestamp and sort
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df = df.sort_values("timestamp")
    
    return df


# checa prepare data for model



In [7]:
new_data=fetch_data_arrangement()
print(new_data)

       bucket           timestamp     value
7626    1_2_2 2024-10-28 10:49:50  corridor
7627    1_2_2 2024-10-28 10:51:05  corridor
7628    1_2_2 2024-10-28 10:51:38  corridor
7629    1_2_2 2024-10-28 10:52:24  corridor
7630    1_2_2 2024-10-28 10:54:01  corridor
...       ...                 ...       ...
17460  1_3_14 2024-11-25 02:39:31      door
7624   1_3_11 2024-11-25 02:39:42  bathroom
17461  1_3_14 2024-11-25 02:39:42      door
7625   1_3_11 2024-11-25 02:39:50  bathroom
17462  1_3_14 2024-11-25 02:39:50      door

[17463 rows x 3 columns]


In [4]:

def data_sorting(df):   

    df_sorted = df.sort_values(by='timestamp')
    df_sorted['human_readable_time'] = pd.to_datetime(df_sorted['timestamp'], unit='ms')
    print(df_sorted.head(40))

    return df_sorted

In [5]:
data_sorted=data_sorting(new_data)

     bucket           timestamp     value human_readable_time
7626  1_2_2 2024-10-28 10:49:50  corridor 2024-10-28 10:49:50
7627  1_2_2 2024-10-28 10:51:05  corridor 2024-10-28 10:51:05
7628  1_2_2 2024-10-28 10:51:38  corridor 2024-10-28 10:51:38
7629  1_2_2 2024-10-28 10:52:24  corridor 2024-10-28 10:52:24
7630  1_2_2 2024-10-28 10:54:01  corridor 2024-10-28 10:54:01
7631  1_2_2 2024-10-28 10:54:19  corridor 2024-10-28 10:54:19
7632  1_2_2 2024-10-28 10:54:55  corridor 2024-10-28 10:54:55
7633  1_2_2 2024-10-28 10:59:15  corridor 2024-10-28 10:59:15
7634  1_2_2 2024-10-28 11:01:20  corridor 2024-10-28 11:01:20
7635  1_2_2 2024-10-28 11:02:20  corridor 2024-10-28 11:02:20
7636  1_2_2 2024-10-30 21:51:19  corridor 2024-10-30 21:51:19
7637  1_2_2 2024-10-30 21:51:33  corridor 2024-10-30 21:51:33
7638  1_2_2 2024-10-30 21:51:45  corridor 2024-10-30 21:51:45
7639  1_2_2 2024-10-30 21:52:01  corridor 2024-10-30 21:52:01
7640  1_2_2 2024-10-30 21:52:11  corridor 2024-10-30 21:52:11
7641  1_

In [6]:

def load_latest_model_minio():
    # Minio Initialize Client
    client = Minio(
        endpoint=MINIO_ENDPOINT, 
        access_key=MINIO_ACCESS_KEY, 
        secret_key=MINIO_SECRET_KEY, 
        secure=False
    )
    base_logger.info("Initialized MinIO client successfully.")

    try:
        # Get the latest model version
        version_file_name = "latest_model_version.txt"
        response = client.get_object(MINIO_BUCKET, version_file_name)
        latest_model_name = response.data.decode("utf-8")
        response.close()
        response.release_conn()
        base_logger.info(f"Latest model version: {latest_model_name}")

        # Load the latest model
        response = client.get_object(MINIO_BUCKET, latest_model_name)
        model_data = response.data.decode("utf-8")
        response.close()
        response.release_conn()
        model = pd.read_json(BytesIO(model_data.encode("utf-8")))
        base_logger.info(f"Loaded model from MinIO: {latest_model_name}")

        return model

    except Exception as e:
        base_logger.error(f"Error loading model from Minio: {e}")
        return None

In [7]:
load_latest_model_minio()

,mean,std
corridor,37394.497021,734453.310073
entrance,17937.270019,114856.892581


In [8]:
#withouy seasonality 
def calculate_stays(df):
    stays = []
    current_stay = None

    for index, row in df.iterrows():
        if current_stay is None:
            current_stay = {
                'room': row['value'],
                'start': row['timestamp'],
                'end': row['timestamp']
            }
        elif row['value'] == current_stay['room']:
            current_stay['end'] = row['timestamp']
        else:
            current_stay['duration'] = (current_stay['end'] - current_stay['start']).total_seconds()
            stays.append(current_stay)
            current_stay = {
                'room': row['value'],
                'start': row['timestamp'],
                'end': row['timestamp']
            }

    if current_stay is not None:
        current_stay['duration'] = (current_stay['end'] - current_stay['start']).total_seconds()
        stays.append(current_stay)

    stays_df = pd.DataFrame(stays)
    print(stays_df)
    return stays_df

stays_df = calculate_stays(data_sorted)

           room               start                 end  duration
0      corridor 2024-10-28 10:49:50 2024-11-06 20:53:37  813827.0
1      bathroom 2024-11-06 20:57:17 2024-11-06 20:57:33      16.0
2      corridor 2024-11-06 20:57:33 2024-11-06 20:57:33       0.0
3      bathroom 2024-11-06 20:57:53 2024-11-06 20:58:14      21.0
4      corridor 2024-11-06 20:58:14 2024-11-06 21:14:15     961.0
...         ...                 ...                 ...       ...
11661      door 2024-11-25 02:39:31 2024-11-25 02:39:31       0.0
11662  bathroom 2024-11-25 02:39:42 2024-11-25 02:39:42       0.0
11663      door 2024-11-25 02:39:42 2024-11-25 02:39:42       0.0
11664  bathroom 2024-11-25 02:39:50 2024-11-25 02:39:50       0.0
11665      door 2024-11-25 02:39:50 2024-11-25 02:39:50       0.0

[11666 rows x 4 columns]


In [23]:

def calculate_seasonal_stays(df):
    # Extract day of the week and hour from the start time
    df['day_of_week'] = df['start'].dt.dayofweek
    df['hour_of_day'] = df['start'].dt.hour

    # Group by room, day of the week, and hour of the day
    grouped = df.groupby(['room', 'day_of_week', 'hour_of_day'])

    # Calculate the number of visits and average duration
    seasonal_stats = grouped['duration'].agg(['count', 'mean']).reset_index()
    seasonal_stats.rename(columns={'count': 'number_of_visits', 'mean': 'average_duration'}, inplace=True)

    # Sort by day of the week and hour of the day
    seasonal_stats = seasonal_stats.sort_values(by=['day_of_week', 'hour_of_day'])

    print(seasonal_stats)
    return seasonal_stats

seasonal_stays_df = calculate_seasonal_stays(stays_df)

         room  day_of_week  hour_of_day  number_of_visits  average_duration
0    bathroom            0            0                15        167.200000
166      door            0            0                14          2.642857
1    bathroom            0            1                 2       1823.500000
167      door            0            1                 2          3.000000
2    bathroom            0            2                 9          1.666667
..        ...          ...          ...               ...               ...
271      door            6           21                40         10.650000
108  bathroom            6           22                32         20.312500
272      door            6           22                32         65.531250
109  bathroom            6           23                40          8.975000
273      door            6           23                40         13.300000

[274 rows x 5 columns]


In [27]:
from datetime import datetime, timedelta

def check_recent_emergency(stays_df, seasonal_stays_df):
    recent_emergency_records = []
    current_time = datetime.now()
    ten_minutes_ago = current_time - timedelta(minutes=10)

    for index, row in stays_df.iterrows():
        if row['start'] >= ten_minutes_ago:
            # Find the corresponding average duration
            avg_duration = seasonal_stays_df[
                (seasonal_stays_df['room'] == row['room']) &
                (seasonal_stays_df['day_of_week'] == row['day_of_week']) &
                (seasonal_stays_df['hour_of_day'] == row['hour_of_day'])
            ]['average_duration'].values

            if len(avg_duration) > 0:
                status = "Normal"
                if row['duration'] > avg_duration[0]:
                    status = "Emergency"
                recent_emergency_records.append({
                    'room': row['room'],
                    'start': row['start'],
                    'duration': row['duration'],
                    'status': status
                })

    recent_emergency_df = pd.DataFrame(recent_emergency_records)
    return recent_emergency_df

# Get the recent emergency status DataFrame
recent_emergency_df = check_recent_emergency(stays_df, seasonal_stays_df)

# Print the recent emergency status DataFrame
print(recent_emergency_df)

Empty DataFrame
Columns: []
Index: []


In [19]:
def check_emergency(stays_df, seasonal_stays_df):
    for index, row in stays_df.iterrows():
        # Find the corresponding average duration
        avg_duration = seasonal_stays_df[
            (seasonal_stays_df['room'] == row['room']) &
            (seasonal_stays_df['day_of_week'] == row['day_of_week']) &
            (seasonal_stays_df['hour_of_day'] == row['hour_of_day'])
        ]['average_duration'].values
        
        if len(avg_duration) > 0 and row['duration'] > avg_duration[0]:
            print(f"Emergency: {row['room']} at {row['start']} with duration {row['duration']} seconds")

check_emergency(stays_df, seasonal_stays_df)

Emergency: bathroom at 2024-11-06 20:57:53 with duration 21.0 seconds
Emergency: corridor at 2024-11-06 20:58:14 with duration 961.0 seconds
Emergency: bed at 2024-11-06 21:17:52 with duration 89.0 seconds
Emergency: bed at 2024-11-06 21:23:26 with duration 83.0 seconds
Emergency: bed at 2024-11-06 21:25:10 with duration 54.0 seconds
Emergency: corridor at 2024-11-06 21:26:55 with duration 1834.0 seconds
Emergency: bathroom at 2024-11-06 22:11:12 with duration 264.0 seconds
Emergency: corridor at 2024-11-06 22:15:55 with duration 495.0 seconds
Emergency: corridor at 2024-11-06 22:25:52 with duration 60.0 seconds
Emergency: bathroom at 2024-11-06 22:27:10 with duration 1181.0 seconds
Emergency: bathroom at 2024-11-06 22:51:27 with duration 143.0 seconds
Emergency: bathroom at 2024-11-06 22:57:38 with duration 372.0 seconds
Emergency: bathroom at 2024-11-06 23:04:03 with duration 273.0 seconds
Emergency: bathroom at 2024-11-06 23:09:31 with duration 1274.0 seconds
Emergency: bathroom at 

In [15]:
def check_emergency(stays_df, seasonal_stays_df):
    for index, row in stays_df.iterrows():
        # Find the corresponding average duration
        avg_duration = seasonal_stays_df[
            (seasonal_stays_df['room'] == row['room']) &
            (seasonal_stays_df['day_of_week'] == row['day_of_week']) &
            (seasonal_stays_df['hour_of_day'] == row['hour_of_day'])
        ]['average_duration'].values
        
        if len(avg_duration) > 0:
            if row['duration'] > avg_duration[0]:
                print(f"Emergency: {row['room']} at {row['start']} with duration {row['duration']} seconds")
            else:
                print(f"Normal: {row['room']} at {row['start']} with duration {row['duration']} seconds")

emergency_status=check_emergency(stays_df, seasonal_stays_df)

Normal: corridor at 2024-10-28 10:49:50 with duration 813827.0 seconds
Normal: bathroom at 2024-11-06 20:57:17 with duration 16.0 seconds
Normal: corridor at 2024-11-06 20:57:33 with duration 0.0 seconds
Emergency: bathroom at 2024-11-06 20:57:53 with duration 21.0 seconds
Emergency: corridor at 2024-11-06 20:58:14 with duration 961.0 seconds
Emergency: bed at 2024-11-06 21:17:52 with duration 89.0 seconds
Normal: corridor at 2024-11-06 21:19:21 with duration 168.0 seconds
Normal: bed at 2024-11-06 21:22:12 with duration 23.0 seconds
Normal: corridor at 2024-11-06 21:22:39 with duration 47.0 seconds
Emergency: bed at 2024-11-06 21:23:26 with duration 83.0 seconds
Normal: corridor at 2024-11-06 21:25:04 with duration 0.0 seconds
Emergency: bed at 2024-11-06 21:25:10 with duration 54.0 seconds
Normal: corridor at 2024-11-06 21:26:09 with duration 0.0 seconds
Normal: bed at 2024-11-06 21:26:10 with duration 0.0 seconds
Normal: corridor at 2024-11-06 21:26:19 with duration 0.0 seconds
Norm

In [17]:
# Convert the emergency status output to a pandas DataFrame
def check_emergency(stays_df, seasonal_stays_df):
    emergency_records = []
    for index, row in stays_df.iterrows():
        # Find the corresponding average duration
        avg_duration = seasonal_stays_df[
            (seasonal_stays_df['room'] == row['room']) &
            (seasonal_stays_df['day_of_week'] == row['day_of_week']) &
            (seasonal_stays_df['hour_of_day'] == row['hour_of_day'])
        ]['average_duration'].values
        
        if len(avg_duration) > 0:
            status = "Normal"
            if row['duration'] > avg_duration[0]:
                status = "Emergency"
            emergency_records.append({
                'room': row['room'],
                'start': row['start'],
                'duration': row['duration'],
                'status': status
            })

    emergency_df = pd.DataFrame(emergency_records)
    return emergency_df

# Get the emergency status DataFrame
emergency_df = check_emergency(stays_df, seasonal_stays_df)

# Save the DataFrame to a text file
emergency_df.to_csv('emergency_status.txt', index=False, sep='\t')